In [1]:
import os
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import myServices as ms
import models as md
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_curve, auc, roc_auc_score, f1_score
import joblib

In [2]:
# to compute ececution time do: 
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Importing and manipulating datasets

In [ ]:
sklearn.metrics.get_scorer_names()

In [3]:
### Cleaning basin1DataSet 
dataSetPath = 'datasets/basin4_RawData.csv'
basinDataSet = pd.read_csv(dataSetPath, index_col = None)
# basin1Light = pd.read_csv('datasetBasin1_NoDataFree.csv', index_col = None)
print(basinDataSet.info())
basinDataSet.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279641 entries, 0 to 279640
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   percentage  15956 non-null   float64
 1   TPI         277252 non-null  float64
 2   TWI         279641 non-null  float64
 3   visibility  279641 non-null  float64
 4   LDSOL5R200  269307 non-null  float64
 5   LDSOL5R150  269307 non-null  float64
 6   LDSOL4R150  264969 non-null  float64
 7   FAProx_025  279641 non-null  float64
 8   FAProx_01   279641 non-null  float64
 9   slope       279636 non-null  float64
 10  elevation   279641 non-null  float64
 11  x_coord     279641 non-null  int64  
 12  y_coord     279641 non-null  int64  
dtypes: float64(11), int64(2)
memory usage: 27.7 MB
None


,percentage,TPI,TWI,visibility,LDSOL5R200,LDSOL5R150,LDSOL4R150,FAProx_025,FAProx_01,slope,elevation,x_coord,y_coord
count,15956.000000,277252.000000,279641.000000,279641.000000,269307.000000,269307.000000,264969.000000,279641.000000,279641.000000,279636.000000,279641.000000,279641.000000,2.796410e+05
mean,4.168714,-0.000226,9.311844,0.061356,0.006653,0.006699,0.003132,272.570003,181.913610,6.353948,136.706610,361059.731023,5.273251e+06
std,1.623044,0.202981,4.026503,0.048661,0.003409,0.004169,0.003029,201.496830,131.952031,5.963830,30.371439,922.890858,1.112358e+03
min,1.000000,-4.801120,-6.874480,0.000050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000390,0.000000,359138.000000,5.270818e+06
25%,5.000000,-0.078290,5.699740,0.027310,0.004060,0.003400,0.000000,110.453610,74.330350,2.285347,129.270280,360253.000000,5.272333e+06
50%,5.000000,-0.000820,10.311960,0.047120,0.006530,0.006400,0.002570,232.594070,156.604600,4.910465,142.805010,361113.000000,5.273358e+06
75%,5.000000,0.078480,12.538330,0.082310,0.009020,0.009550,0.005360,395.000000,268.700590,8.516732,153.404530,361838.000000,5.274063e+06
max,5.000000,4.880940,22.801570,0.448310,0.019030,0.021440,0.012770,975.140990,675.740360,67.607030,216.042620,362883.000000,5.275618e+06


In [21]:
basinDataSet.isna().any()

percentage    False
TPI           False
TWI           False
visibility    False
LDSOL5R200    False
LDSOL5R150    False
LDSOL4R150    False
FAProx_025    False
FAProx_01     False
slope         False
elevation     False
x_coord       False
y_coord       False
dtype: bool

In [12]:
basinDataSet['percentage'].fillna(0,inplace=True)

In [13]:
basinDataSet.dropna(subset=['TPI'],inplace=True)

In [22]:
basinDataSet.isna().sum()

percentage    0
TPI           0
TWI           0
visibility    0
LDSOL5R200    0
LDSOL5R150    0
LDSOL4R150    0
FAProx_025    0
FAProx_01     0
slope         0
elevation     0
x_coord       0
y_coord       0
dtype: int64

In [ ]:
basinDataSet.dropna(inplace=True)

In [67]:
basinDataSet.isna().sum()
# print(basinDataSet['FAcc'].max())

percentage    0
TWI           0
TPI           0
LDSOL3R150    0
LDSOL4R150    0
LDSOL4R200    0
FAProx_01     0
FAProx_025    0
visibility    0
slope         0
elevation     0
x_coord       0
y_coord       0
dtype: int64

In [ ]:
#### NOrmalize Flow Accumulation
basinDataSet['FAcc'] = (basinDataSet['FAcc']- basinDataSet['FAcc'].min())/(basinDataSet['FAcc'].max()-basinDataSet['FAcc'].min())


In [ ]:
### Replacing with 0 
repalcer  = basinDataSet['FAProx_01'].to_numpy()
basinDataSet['FAProx_01'] = [0 if repalcer[j] == -9999 else repalcer[j] for j in range(len(repalcer))]                                                                                                                         
                                                                                                                          

In [26]:
## Transform a column datatype
repalcer  = basinDataSet['percentage'].to_numpy().astype('int')
basinDataSet.loc[:,'percentage'] = repalcer

In [23]:
basinDataSet.describe()

,percentage,TPI,TWI,visibility,LDSOL5R200,LDSOL5R150,LDSOL4R150,FAProx_025,FAProx_01,slope,elevation,x_coord,y_coord
count,277252.000000,277252.000000,277252.000000,277252.000000,277252.000000,277252.000000,277252.000000,277252.000000,277252.000000,277252.000000,277252.000000,277252.000000,2.772520e+05
mean,0.239677,-0.000226,9.322267,0.060643,0.006447,0.006494,0.002990,271.329211,180.848976,6.362444,136.693555,361058.502016,5.273243e+06
std,1.045516,0.202981,4.026137,0.047200,0.003559,0.004272,0.003032,201.201950,131.322930,5.969607,30.189266,919.139056,1.110446e+03
min,0.000000,-4.801120,-6.874480,0.000050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000390,0.454800,359148.000000,5.270818e+06
25%,0.000000,-0.078290,5.709468,0.027240,0.003770,0.003080,0.000000,110.000000,74.330350,2.282698,129.274703,360258.000000,5.272328e+06
50%,0.000000,-0.000820,10.316390,0.046935,0.006400,0.006220,0.002260,230.488600,155.563490,4.922205,142.743010,361113.000000,5.273353e+06
75%,0.000000,0.078480,12.549648,0.081700,0.008920,0.009450,0.005250,392.460170,266.833280,8.533800,153.260227,361833.000000,5.274053e+06
max,5.000000,4.880940,22.760400,0.448310,0.019030,0.021440,0.012770,975.140990,670.913570,67.607030,216.042620,362868.000000,5.275608e+06


In [27]:
basinDataSet.to_csv('datasets/basin4_CleanDataSet.csv', index=None)

In [28]:
### Making quadratic transformation in Labels for regression
DS = pd.read_csv('datasets/basin4_CleanDataSet.csv', index_col = None)
print(Counter(DS['percentage']))

# y_Quad = md.quadraticRechapeLabes(DS['percentage'], -0.125, 0.825)
# DS['percentage'] = y_Quad
# print(Counter(DS['percentage']))
# DS.to_csv('basin1TrainUnbalanced_QuadTarget.csv', index = None)

Counter({0: 261309, 5: 12627, 1: 3316})


In [17]:
ds = DS.head(5)
s = {}
s['Datas'] = ds
print(s)

{'Datas':    percentage      TPI       TWI  visibility  LDSOL5R200  LDSOL5R150  \
0         0.0 -0.05424  11.17568     0.26287         0.0         0.0   
1         0.0  0.19205  10.12663     0.27911         0.0         0.0   
2         0.0  0.09936  10.32567     0.27625         0.0         0.0   
3         0.0  0.06192   5.88959     0.10397         0.0         0.0   
4         0.0 -0.04243  11.30930     0.08822         0.0         0.0   

   LDSOL4R150  FAProx_025  FAProx_01    slope  elevation  x_coord  y_coord  
0         0.0   443.22681  440.36914  0.56349  128.15756   359173  5274413  
1         0.0   438.32065  435.48825  0.75803  128.33572   359178  5274413  
2         0.0   423.61539  420.86218  1.58734  128.00537   359193  5274413  
3         0.0   418.71829  415.99277  2.55058  127.80929   359198  5274413  
4         0.0   413.82361  411.12653  2.13726  127.53185   359203  5274413  }


## balanced sampling

In [29]:
## Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

X,Y = ms.importDataSet('datasets/basin4_CleanDataSet.csv', 'percentage')
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=50)
for train_index, test_index in sss.split(X, Y):
    print("TRAIN:", train_index.size, "TEST:", test_index.size)
    X_train = X.iloc[train_index]
    y_train = Y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = Y.iloc[test_index]

TRAIN: 221801 TEST: 55451


In [30]:
## Describing training set
print(len(X_train['elevation']), len(y_train) )
Counter(y_train)

221801 221801


Counter({0: 209047, 5: 10101, 1: 2653})

In [32]:
#####    Creating training set     #####
X_train.loc[:,'percentage'] = y_train
print(X_train.head())
## Removing coordinates from training set
X_train.drop(['x_coord','y_coord'], axis =1, inplace=True)
X_train.head()
Counter(X_train['percentage'])

            TPI       TWI  visibility  LDSOL5R200  LDSOL5R150  LDSOL4R150  \
251774 -0.00034  14.39598     0.01397     0.00774     0.01004     0.00474   
160243 -0.21971  13.37331     0.10493     0.00310     0.00114     0.00000   
27303   0.00001  15.71532     0.01069     0.00544     0.00800     0.00197   
189468 -0.24402   9.40016     0.04387     0.00712     0.00975     0.00550   
265065  0.06554   5.30098     0.18383     0.00202     0.00269     0.00000   

        FAProx_025  FAProx_01     slope  elevation  x_coord  y_coord  \
251774   255.04901   76.32169   0.63286  143.95445   361653  5271843   
160243   276.81219  276.81219   6.91255  149.28308   360853  5273343   
27303    325.96011  111.01802   0.01081  122.72339   360073  5273768   
189468     7.07107    5.00000  12.48343  139.97217   361783  5272973   
265065   822.51135  428.51486   2.63780  183.58789   359913  5271303   

        percentage  
251774           0  
160243           0  
27303            5  
189468           0  

/var/folders/s5/cqkwnnps0f1_35_bccj0y8q80000gn/T/ipykernel_36263/424696301.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'percentage'] = y_train


Counter({0: 209047, 5: 10101, 1: 2653})

/var/folders/s5/cqkwnnps0f1_35_bccj0y8q80000gn/T/ipykernel_36263/1019043263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['x_coord','y_coord'], axis =1, inplace=True)


,TPI,TWI,visibility,LDSOL5R200,LDSOL5R150,LDSOL4R150,FAProx_025,FAProx_01,slope,elevation,percentage
251774,-0.00034,14.39598,0.01397,0.00774,0.01004,0.00474,255.04901,76.32169,0.63286,143.95445,0
160243,-0.21971,13.37331,0.10493,0.00310,0.00114,0.00000,276.81219,276.81219,6.91255,149.28308,0
27303,0.00001,15.71532,0.01069,0.00544,0.00800,0.00197,325.96011,111.01802,0.01081,122.72339,5
189468,-0.24402,9.40016,0.04387,0.00712,0.00975,0.00550,7.07107,5.00000,12.48343,139.97217,0
265065,0.06554,5.30098,0.18383,0.00202,0.00269,0.00000,822.51135,428.51486,2.63780,183.58789,0


In [35]:
X_train.to_csv('datasets/basin4_Training.csv', index=None)

In [36]:
#####. Creating training set
print(X_test.head())
X_test.loc[:,'percentage'] = y_test
print(X_test.head())
print(X_test.info())
Counter(X_test['percentage'])


            TPI       TWI  visibility  LDSOL5R200  LDSOL5R150  LDSOL4R150  \
8972    0.36684   6.83490     0.06692     0.00889     0.00573     0.00043   
270688  0.09789  12.68315     0.20336     0.00152     0.00001     0.00000   
91516  -0.25639   9.15047     0.01869     0.00978     0.01067     0.00550   
115409  0.06341   3.66044     0.07109     0.00868     0.00885     0.00484   
214249  0.05304   4.62529     0.06147     0.00702     0.00338     0.00128   

        FAProx_025  FAProx_01     slope  elevation  x_coord  y_coord  
8972     141.50972  141.50972   9.06672  130.94017   360243  5274248  
270688   876.81244  434.10828   7.19772  174.65186   359968  5271153  
91516    107.00467  107.00467  13.66632   63.74500   359728  5274558  
115409    55.00000   55.00000   0.38351  125.55405   360913  5273953  
214249   228.08989  156.60460   2.54051  153.73796   361913  5272473  
            TPI       TWI  visibility  LDSOL5R200  LDSOL5R150  LDSOL4R150  \
8972    0.36684   6.83490     0.06

/var/folders/s5/cqkwnnps0f1_35_bccj0y8q80000gn/T/ipykernel_36263/1397829333.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:,'percentage'] = y_test


Counter({0: 52262, 5: 2526, 1: 663})

In [37]:
X_test.to_csv('datasets/basin4_Test.csv', index=None)

In [38]:
## This proportions are the reason why a sample_weight of 0.01 for the majority class give best results for regression
totalTrain = sum([209047, 10101, 2653])
totalValidation = sum([52262, 2526, 663])
print("Summary of traning and test dataset class balance")
print(f"Training Set:", '\n', "Class 0: %.3f" %(209047/totalTrain), " Class 1: %.4f" %(2653/totalTrain), "Class 5: %.4f"%(10101/totalTrain))
print("Testing Set:", '\n', "Class 0: %.3f" %(52262/totalValidation)," Class 1: %.4f" %(663/totalValidation),  "Class 5: %.4f"%(2526/totalValidation))



Summary of traning and test dataset class balance
Training Set: 
 Class 0: 0.942  Class 1: 0.0120 Class 5: 0.0455
Testing Set: 
 Class 0: 0.942  Class 1: 0.0120 Class 5: 0.0456


In [40]:
print(totalTrain, totalValidation)

221801 55451


In [ ]:
model = ms.loadModel('./outputs/2022-08-05/00-35-58/2208050035.pkl')
dataSetToSave = ms.makePredictionToImportAsSHP(csvName, model, X, Y, 'percentage')
print(dataSetToSave.head())

## Controled sampling

In [ ]:
DS = pd.read_csv('basin1Light_Clean_Training.csv', index_col = None)
print(DS.head())


In [ ]:
print(DS.columns)

In [ ]:
plt.boxplot(DS['FAcc']) # , , DS['elevation'], DS['disToRiv']]

In [ ]:
## Resampling appliying class selection by rule:

# RULE1: Select point at a distance to river less than 300m. 

# # newDS = pseudoClassCreation(DS, "distanceToRiver", 300, 2)
def pseudoClassCreation(dataset, conditionVariable, threshold, pseudoClass, targetClassName):
    '''
    Replace <targetClass> by  <pseudoClass> where <conditionVariable >= threshold>. 
    Return:
      dataset with new classes group. 
    '''
    datsetReclassified = dataset.copy()
    actualTarget = (np.array(dataset[targetClassName])).ravel()
    conditionVar = (np.array(dataset[conditionVariable])).ravel()
    datsetReclassified[targetClassName] = [ pseudoClass if conditionVar[j] >= threshold 
                                           else actualTarget[j]
                                           for j in range(len(actualTarget))]
    print(Counter(datsetReclassified[targetClassName]))
    return  datsetReclassified

def revertPseudoClassCreation(dataset, originalClass, pseudoClass, targetClassName):
    '''
    Restablich  <targetClass> with <originalClass> where <targetClassName == pseudoClass>. 
    Return:
      dataset with original classes group. 
    '''
    datsetReclassified = dataset.copy()
    actualTarget = (np.array(dataset[targetClassName])).ravel()
    datsetReclassified[targetClassName] = [ originalClass if actualTarget[j] == pseudoClass
                                           else actualTarget[j]
                                           for j in range(len(actualTarget))]
    print(Counter(datsetReclassified[targetClassName]))
    return  datsetReclassified


print(Counter(X_train['percentage']))
newDS = pseudoClassCreation(X_train, 'disToRiv', 200, 2, 'percentage')
y = newDS['percentage']
newDS.drop(['percentage'], axis=1, inplace = True)
x_res,y_res = ms.randomUndersampling(newDS, y, )
x_res['percentage'] = y_res
# newDatase = revertPseudoClassCreation(x_res, 0, 2, 'percentage')


In [ ]:
x_res.to_csv('basin1ControlClass0Sampling4Class_ToSHP.csv',index = None)

# Data description and visualization

In [ ]:
#### import dataset to describe

DS= pd.read_csv('datasets/basin1Light_Clean.csv', index_col=None)
DS.head()

In [ ]:
DS.drop(['x_coord','y_coord'], axis = 1, inplace=True)
DS.head()

In [ ]:
### FAcc vs Labels
targets = DS['percentage']
FAcc = original['FAcc']
FAcc_norm = DS['FAcc_norm']
fig, axs = plt.subplots(1, 2, figsize=(13,4), sharey=True)
fig.text(-0.02, 0.5, 'labels', va='center', rotation='vertical')
fig.text(0.5, 1, 'Flow accumulation vs labels distribution', ha ='center')
axs[0].scatter(FAcc,targets)
# axs[0].set_title("Facc")
axs[0].set(xlabel='a) Flow Accumulation')
axs[1].scatter(FAcc_norm,targets)
# axs[1].set_title("FAcc_norm")
axs[1].set(xlabel='b) Flow Accumulation estandardized')
plt.rcParams['font.size'] = '20'
fig.tight_layout()


In [ ]:

## Plot all features vs labels
# 'disToRiv', 'TWI', 'TPI', 'slope', 'elevation',

targets = DS['percentage']
# targets = np.where(targets == 5,2,targets)

E = DS['elevation'] 
slope = DS['slope']
FAcc = DS['FAcc']
TWI = DS['TWI']
TPI = DS['TPI']
DLSOL4R150 = DS['DLSOL4R150']
DLSOL5R150 = DS['DLSOL5R150']
DLSOL5R200 = DS['DLSOL5R200']
FAProx_01 = DS['FAProx_01']
FAProx_025 = DS['FAProx_025']
visibility = DS['visibility']

fig, axs = plt.subplots(4,3, figsize=(13, 8), sharey=True)
fig.supylabel('Labels')
plt.rcParams['font.size'] = '15'
plt.yticks([0,1,5])

'''
E = DS['elevation'] 
slope = DS['slope']
FAcc = DS['FAcc']
TWI = DS['TWI']
'''
axs[0, 0].scatter(E,targets)
axs[0, 0].set_title("Elevation")
axs[1, 0].scatter(slope,targets)
axs[1, 0].set_title("Slope")
axs[2, 0].scatter(FAcc,targets)
axs[2, 0].set_title("Flow accumulation")
axs[3, 0].scatter(TWI,targets)
axs[3, 0].set_title("TWI")

'''
TPI = DS['TPI']
DLSOL4R150 = DS['DLSOL4R150']
DLSOL5R150 = DS['DLSOL5R150']
DLSOL5R200 = DS['DLSOL5R200']
'''
axs[0, 1].scatter(TPI,targets)
axs[0, 1].set_title('TPI')
axs[1, 1].scatter(DLSOL4R150,targets)
axs[1, 1].set_title("DLSOL4R150")
axs[2, 1].scatter(DLSOL5R150,targets)
axs[2, 1].set_title("DLSOL5R150")
axs[3, 1].scatter(DLSOL5R200,targets)
axs[3, 1].set_title("DLSOL5R200")

'''
FAProx_01 = DS['FAProx_01']
FAProx_025 = DS['FAProx_025']
visibility = DS['visibility']
'''
axs[0, 2].scatter(FAProx_01,targets)
axs[0, 2].set_title('FAProx_01')
axs[1, 2].scatter(FAProx_025,targets)
axs[1, 2].set_title("FAProx_025")
axs[2, 2].scatter(visibility,targets)
axs[2, 2].set_title("Visibility")

fig.tight_layout()


In [ ]:
print(DS.head())
#  Return a dataset with the rows corresponding to the index where condition in DS.columName is valid. 
dsArray = DS[DS.percentage != 0] print(dsArray.head())
print(dsArray.head()) 

In [ ]:
sns.set(font_scale=1.5)
sns.pairplot(DS, hue = 'percentage', diag_kind = 'kde', 
             plot_kws = {'alpha': 0.8, 's': 100},
             height = 4, corner=True, palette = "Set2")# vars = ['life_exp', 'log_pop', 'log_gdp_per_cap'],

# sns.pairplot(DS, hue="percentage")

In [ ]:
####. Covariance Matrix
sns.set(font_scale=0.7)
matrix = DS.corr().round(2)
sns.heatmap(matrix, annot=True)
plt.set_figsize=(25,20)
plt.show()

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(criterion='entropy', random_state = 50)
x_train,y_train = ms.importDataSet('basin1Train.csv', 'percentage')
classifier = OneVsRestClassifier(estimator).fit(x_train,y_train)


In [ ]:
classifier = ms.loadModel('./outputs/2022-08-05/11-01-57/2208051101.pkl')
x_test,y_test = ms.importDataSet('basin1Test.csv', 'percentage')

x_test = ms.removeCoordinatesFromDataSet(x_test)

# y_prob = classifier.predict_proba(x_test)
#print(np.unique(y_prob))

In [ ]:
md.plot_ROC_AUC_OneVsRest(classifier, x_test, y_test)

In [ ]:
_,y_test = ms.importDataSet('./bestModels/Classifier/10-18-08/2208051018prediction_basin1Test.csv', 'prediction')
unique, count = np.unique(y_test, return_counts=True)
total = count.sum()
print(total)
percent = np.round(np.zeros_like(unique).astype('float16'),3)
print('values, counts , percent')
for i in range(len(unique)):    
   percent[i] = (count[i]/total)*100
   print(unique[i],"\t", count[i], percent[i])
